# Project Data Analytics: Predicting BikeShare Rentals: Seasonality & Weather (2011-2012)

- Name: Stefanus Bernard Melkisedek
- Email: stefanussipahutar@gmail.com
- Id Dicoding: stefansphtr

## Define Business Questions

| **NO** | **BUSINESS QUESTION**                                                                                                                  |
| :----: | -------------------------------------------------------------------------------------------------------------------------------------- |
|   1.   | What is the impact of temperature on bike rentals?                              |
|   2.   | In which specific seasons do we see the most significant shifts in rental volume? |
|   3.   | Are there weekday-specific events that significantly affect rental volume in our area?                                                                                  |
|   4.   | Beyond temperature, which specific weather conditions (e.g., rain, wind) have the most significant impact on rentals?                                                  |
|   5.   | Based on historical data, can we develop a reliable model to predict daily/weekly bike rental demand?                                                     |

## Prepare all the library needed

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from module.plot_missing_value import percentage as pmv